In [ ]:
# Author: Ryan Ku

import pandas as pd
import numpy as np
from datetime import date
import time
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
# locations that aren't counties but can be treated as one for these purposes
# EXCEPTIONS = ["District of Columbia"]
EXCEPTIONS = []

# returns a partial dataframe filtered by col:text (key:value) pair in filter_dict (i.e. removes all rows that don't satisfy filter_dict)
# if remove_dict is specified, this function removes rows that satisfy remove_dict, BUT keeps any that also satisfies filter_dict
def filter_by_value(df, filter_dict: dict = {}, remove_dict: dict = {}):
    # if remove_dict was specified, remove rows that satisfy remove_dict BUT not filter_dict (i.e. also keep everything satisfied by filter_dict):
    i_remove = set()
    i_keep = set()
    if len(remove_dict) > 0:
        for col, text in remove_dict.items():
            if text == "NaN":
                i_remove = i_remove.union(set(df[df[col].isna()].index))
            else:
                # if is list instead, run through list
                if type(text) is list:
                    for item in text:
                        # if is int/float instead, check integer value
                        if type(item) is int or type(item) is float:
                            i_remove = i_remove.union(set(df[df[col] == item].index))
                        else:
                            i_remove = i_remove.union(set(df[df[col].str.contains(item)].index))
                # if is int/float instead, check integer value
                elif type(text) is int or type(text) is float:
                    i_remove = i_remove.union(set(df[df[col] == text].index))
                else:
                    i_remove = i_remove.union(set(df[df[col].str.contains(text)].index))
        for col, text in filter_dict.items():
            if text == "NaN":
                i_keep = i_keep.union(set(df[df[col].isna()].index))
            else:
                # if is list instead, run through list
                if type(text) is list:
                    for item in text:
                        # if is int/float instead, check integer value
                        if type(item) is int or type(item) is float:
                            i_keep = i_keep.union(set(df[df[col] == item].index))
                        else:
                            i_keep = i_keep.union(set(df[df[col].str.contains(item)].index))
                        
                # if is int/float instead, check integer value
                elif type(text) is int or type(text) is float:
                    i_keep = i_keep.union(set(df[df[col] == text].index))
                else:
                    i_keep = i_keep.union(set(df[df[col].str.contains(text)].index))
        return df.drop(index = i_remove - i_keep)
    # if remove_dict was not specified, filter rows by filter_dict:
    else:
        data = df.copy()
        for col, text in filter_dict.items():
            if text == "NaN":
                i_remove = i_remove.union(set(data[~data[col].isna()].index))
            else:
                # if is list instead, run through list
                if type(text) is list:
                    for item in text:
                        # if is int/float instead, check integer value
                        if type(item) is int or type(item) is float:
                            i_keep = i_keep.union(set(data[data[col] == item].index))
                        else:
                            i_keep = i_keep.union(set(data[data[col].str.contains(item)].index))
                        
                # if is int/float instead, check integer value
                elif type(text) is int or type(text) is float:
                    i_remove = i_remove.union(set(data[~(data[col] == text)].index))
                else:
                    i_remove = i_remove.union(set(data[~data[col].str.contains(text)].index))

            data.drop(index=i_remove, inplace=True)
            i_remove = set()
        if len(i_keep) > 0:
            return data.drop(index = set(data.index) - i_keep)
        return data

# removes all rows that don't correspond to individual counties (only works on USDoA datasets)
# NOTE: boroughs are ignored for sake of simplicity (there's only ~15-20 anyways)
def filter_counties(df, exceptions: list = []):
    # remove rows that don't contain the word "County" in the "County" column
    if len(EXCEPTIONS) > 0:
        i_remove = df[~df["County"].str.contains("County") & ~df["County"].str.contains("|".join(EXCEPTIONS))].index
    else:
        i_remove = df[~df["County"].str.contains("County")].index
    return df.drop(index = i_remove)

# return true if NAs found under specific column
def any_na(df, col) -> bool:
    return df[col].isnull().values.any()

In [ ]:
# retrieves relevant virus data
def get_virus_data():
    dtype = {
        "fips" : "Int32",
        "cases" : "Int32",
        "deaths" : "Int32"
    }
    columns = {
        "date" : "Date",
        "county" : "County",
        "state" : "State",
        "fips" : "FIPS",
        "cases" : "Cases",
        "deaths" : "Deaths"
    }
    data = pd.read_csv("./datasets/NYTimes/us-counties_2020_6_3.csv", sep=",", header=0, engine="python", dtype=dtype, parse_dates=["date"]).rename(columns=columns)

    # remove data from unknown locations, BUT keep Puerto Rico data
    data = filter_by_value(data, filter_dict={"State": "Puerto Rico"}, remove_dict={"County": "Unknown"})
    # add Day column to track number of days since start of data
    data["Day"] = (data["Date"] - data["Date"][0]).dt.days
    # TODO: convert state to state abbreviation
    data.fillna(value={"FIPS": -1}, inplace=True)
    return data

# retrieves relevant education data
def get_education_data():
    dtype = {
        "FIPS Code" : "Int32",
        "Less than a high school diploma, 2014-18" : "Int32",
        "High school diploma only, 2014-18" : "Int32",
        "Some college or associate's degree, 2014-18" : "Int32",
        "Bachelor's degree or higher, 2014-18" : "Int32",
    }
    columns = {
        "FIPS Code" : "FIPS",
        "State" : "State",
        "Area name" : "County",
        "Less than a high school diploma, 2014-18" : "Less_HS",
        "High school diploma only, 2014-18" : "HS_only",
        "Some college or associate's degree, 2014-18" : "College_Associate",
        "Bachelor's degree or higher, 2014-18" : "Bachelors",
        "Percent of adults with less than a high school diploma, 2014-18" : "PCT_Less_HS",
        "Percent of adults with a high school diploma only, 2014-18" : "PCT_HS_only",
        "Percent of adults completing some college or associate's degree, 2014-18" : "PCT_College_Associate",
        "Percent of adults with a bachelor's degree or higher, 2014-18" : "PCT_Bachelors"
    }
    data = pd.read_csv("./datasets/USDoA/Education_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    return filter_counties(data)

# retrieves relevant population data
def get_population_data():
    dtype = {
        "FIPS" : "Int32",
        "CENSUS_2010_POP" : "Int32",
        "POP_ESTIMATE_2018" : "Int32",
        "Births_2018" : "Int32",
        "Deaths_2018" : "Int32",
        "INTERNATIONAL_MIG_2018" : "Int32",
        "DOMESTIC_MIG_2018" : "Int32",
        "NET_MIG_2018" : "Int32",
        "GQ_ESTIMATES_2018" : "Int32"
    }
    columns = {
        "FIPS" : "FIPS",
        "State" : "State",
        "Area_Name" : "County",
        "CENSUS_2010_POP" : "CENSUS_2010_POP",                     # 4/1/2010 resident total Census 2010 population
        "POP_ESTIMATE_2018" : "POP_ESTIMATE_2018",                 # 7/1/2018 resident total population estimate
        "Births_2018" : "BIRTHS_2018",                             # Births in period 7/1/2017 to 6/30/2018
        "Deaths_2018" : "DEATHS_2018",                             # Deaths in period 7/1/2017 to 6/30/2018
        "INTERNATIONAL_MIG_2018" : "INTERNATIONAL_MIG_2018",       # Net international migration in period 7/1/2017 to 6/30/2018
        "DOMESTIC_MIG_2018" : "DOMESTIC_MIG_2018",                 # Net domestic migration in period 7/1/2017 to 6/30/2018
        "NET_MIG_2018" : "NET_MIG_2018",                           # Net migration in period 7/1/2017 to 6/30/2018
        "GQ_ESTIMATES_2018" : "GQ_ESTIMATES_2018",                 # 7/1/2018 Group Quarters total population estimate
        "R_birth_2018" : "R_BIRTHS_2018",                          # Birth rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_death_2018" : "R_DEATHS_2018",                          # Death rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_INTERNATIONAL_MIG_2018" : "R_INTERNATIONAL_MIG_2018",   # Net international migration rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_DOMESTIC_MIG_2018" : "R_DOMESTIC_MIG_2018",             # Net domestic migration rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_NET_MIG_2018" : "R_NET_MIG_2018"                        # Net migration rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
    }
    data = pd.read_csv("./datasets/USDoA/PopulationEstimates_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    return filter_counties(data)

# retrieves relevant poverty data
def get_poverty_data():
    dtype = {
        "FIPStxt" : "Int32",
        "POVALL_2018" : "Int32",
        "POV017_2018" : "Int32",
        "POV517_2018" : "Int32",
        "MEDHHINC_2018" : "Int32"
    }
    columns = {
        "FIPStxt" : "FIPS",
        "Stabr" : "State",
        "Area_name" : "County",
        "POVALL_2018" : "Poverty_Total",               # total amount in poverty in 2018
        "PCTPOVALL_2018" : "PCT_Poverty_Total",        # percentage in poverty in 2018
        "POV017_2018" : "Poverty_0_17",                # amount in poverty ages 0-17 in 2018
        "PCTPOV017_2018" : "PCT_Poverty_0_17",         # percentage in poverty ages 0-17 in 2018
        "POV517_2018" : "Poverty_5_17",                # amount in poverty ages 5-17 in 2018
        "PCTPOV517_2018" : "PCT_Poverty_5_17",         # percentage in poverty ages 5-17 in 2018
#         "MEDHHINC_2018" : "Median_Household_Income"    # median household income in 2018    NOTE: repetitive?
    }
    data = pd.read_csv("./datasets/USDoA/PovertyEstimates_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    return filter_counties(data)

# retrieves relevant unemployment data
def get_unemployment_data():
    dtype = {
        "FIPS" : "Int32",
        "Civilian_labor_force_2018" : "Int32",
        "Employed_2018" : "Int32",
        "Unemployed_2018" : "Int32",
    }
    columns = {
        "FIPS" : "FIPS",
        "State" : "State",
        "Area_name" : "County",
        "Civilian_labor_force_2018" : "Total_Labor_Force",
        "Employed_2018" : "Employed",
        "Unemployed_2018" : "Unemployed",
        "Unemployment_rate_2018" : "PCT_Unemployed",
        "Median_Household_Income_2018" : "Median_Household_Income",
        "Med_HH_Income_Percent_of_State_Total_2018" : "Median_Household_Income_County_to_State_Ratio"
    }
    data = pd.read_csv("./datasets/USDoA/Unemployment_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    # remove non-counties
    data = filter_counties(data)
    # strip and convert "Median_Household_Income" column to int
    data["Median_Household_Income"] = data["Median_Household_Income"].astype(str).map(lambda x: x.lstrip("$").rstrip(" ").replace(",", "")).astype(int)
    return data
    
# retrieves relevant land area data
def get_land_data():
    columns = {
        "Areaname" : "Areaname",       # County, State
        "STCOU" : "FIPS",              # FIPS
        "LND110210D" : "Land_Area",    # -> Land area in square miles 2010
        "LND210200D" : "Water_Area"    # -> Water area in square miles 2000
    }
    data = pd.read_csv("./datasets/USCensus/LND01_2020_5_21.csv", sep=",", header=0, engine="python", usecols=columns.keys()).rename(columns=columns)
    return data

# retrieves relevant housing data
def get_housing_data():
    dtype = {
        "STCOU" : "Int32",
        "HSG030210D" : "Int32",
        "HSG171209D" : "Int32",
        "HSG172209D" : "Int32",
        "HSG173209D" : "Int32",
        "HSG174209D" : "Int32",
        "HSG175209D" : "Int32",
        "HSG176209D" : "Int32",
        "HSG177209D" : "Int32",
        "HSG178209D" : "Int32",
        "HSG179209D" : "Int32"
    }
    columns = {
        "Area_name" : "Areaname",         # County, State
        "STCOU" : "FIPS",                 # FIPS
        "HSG030210D" : "Total_Units",     # Total housing units 2010 (complete count)
        "HSG171209D" : "1_Room",          # Housing units with 1 room, 2005-2009
        "HSG172209D" : "2_Rooms",	      # Housing units with 2 rooms, 2005-2009
        "HSG173209D" : "3_Rooms",	      # Housing units with 3 rooms, 2005-2009
        "HSG174209D" : "4_Rooms",	      # Housing units with 4 rooms, 2005-2009
        "HSG175209D" : "5_Rooms",	      # Housing units with 5 rooms, 2005-2009
        "HSG176209D" : "6_Rooms",	      # Housing units with 6 rooms, 2005-2009
        "HSG177209D" : "7_Rooms",	      # Housing units with 7 rooms, 2005-2009
        "HSG178209D" : "8_Rooms",	      # Housing units with 8 rooms, 2005-2009
        "HSG179209D" : "9+_Rooms",	      # Housing units with 9 rooms or more, 2005-2009
        "HSG180209D" : "Median_Rooms"     # Housing units - median rooms, 2005-2009
    }
    data = pd.read_csv("./datasets/USCensus/HSG_modified.csv", sep=",", header=0, engine="python", dtype=dtype, thousands=",").rename(columns=columns)
    return data

# retrieves relevant age data
def get_age_data():
    dtype = {
        "STCOU" : "Int32",
        "AGE290209D" : "Int32",
        "AGE130209D" : "Int32",
        "AGE160209D" : "Int32",
        "AGE230209D" : "Int32",
        "AGE260209D" : "Int32",
        "AGE340209D" : "Int32",
        "AGE370209D" : "Int32",
        "AGE430209D" : "Int32",
        "AGE460209D" : "Int32",
        "AGE530209D" : "Int32",
        "AGE560209D" : "Int32",
        "AGE630209D" : "Int32",
        "AGE660209D" : "Int32",
        "AGE690209D" : "Int32",
        "AGE730209D" : "Int32",
        "AGE800209D" : "Int32",
        "AGE830209D" : "Int32",
        "AGE870209D" : "Int32",
        "AGE900209D" : "Int32"
    }
    columns = {
        "Areaname" : "Areaname",              # County, State
        "STCOU" : "FIPS",                     # FIPS
        "AGE050200D" : "Median_Age",          # Resident population: Median age (April 1 - complete count) 2000
        "AGE290209D" : "Population_0_18",     # Resident population under 18 years (July 1 - estimate) 2009
        "AGE130209D" : "Population_0_5",      # Resident population under 5 years (July 1 - estimate) 2009
        "AGE160209D" : "Population_5_9",      # Resident population 5 to 9 years (July 1 - estimate) 2009
        "AGE230209D" : "Population_10_14",    # Resident population 10 to 14 years (July 1 - estimate) 2009
        "AGE260209D" : "Population_15_19",    # Resident population 15 to 19 years (July 1 - estimate) 2009
        "AGE340209D" : "Population_20_24",    # Resident population 20 to 24 years (July 1 - estimate) 2009
        "AGE370209D" : "Population_25_29",    # Resident population 25 to 29 years (July 1 - estimate) 2009
        "AGE430209D" : "Population_30_34",    # Resident population 30 to 34 years (July 1 - estimate) 2009
        "AGE460209D" : "Population_35_39",    # Resident population 35 to 39 years (July 1 - estimate) 2009
        "AGE530209D" : "Population_40_44",    # Resident population 40 to 44 years (July 1 - estimate) 2009
        "AGE560209D" : "Population_45_49",    # Resident population 45 to 49 years (July 1 - estimate) 2009
        "AGE630209D" : "Population_50_54",    # Resident population 50 to 54 years (July 1 - estimate) 2009
        "AGE660209D" : "Population_55_59",    # Resident population 55 to 59 years (July 1 - estimate) 2009
        "AGE690209D" : "Population_60_64",    # Resident population 60 to 64 years (July 1 - estimate) 2009
        "AGE730209D" : "Population_65_69",    # Resident population 65 to 69 years (July 1 - estimate) 2009
        "AGE800209D" : "Population_70_74",    # Resident population 70 to 74 years (July 1 - estimate) 2009
        "AGE830209D" : "Population_75_79",    # Resident population 75 to 79 years (July 1 - estimate) 2009
        "AGE870209D" : "Population_80_84",    # Resident population 80 to 84 years (July 1 - estimate) 2009
        "AGE900209D" : "Population_85+"       # Resident population 85 years and over (July 1 - estimate) 2009
    }
    data = pd.read_csv("./datasets/USCensus/AGE_modified.csv", sep=",", header=0, engine="python", dtype=dtype, thousands=",").rename(columns=columns)
    # make PCT columns for relative population diversity
    # 0. list columns from age groups 0 to 85+
#     population_columns = ["Population_0_5", "Population_5_9", "Population_10_14", "Population_15_19", "Population_20_24", "Population_25_29", "Population_30_34",\
#                "Population_35_39", "Population_40_44", "Population_45_49", "Population_50_54", "Population_55_59", "Population_60_64", "Population_65_69",\
#                "Population_70_74", "Population_75_79", "Population_80_84", "Population_85+"]
    population_columns = list(columns.values())[4:]
    # 1. get total calculated population for each county
    total_population = data[population_columns].sum(axis="columns")
    # 2. PCT = age group population / total population
    population_columns.append("Population_0_18")
    for column in population_columns:
        data["PCT_" + column] = data[column] / total_population
    
    return data

get_age_data()

# retrieves relevant race data
def get_race_data():
    dtype = {
        "STCOU" : "Int32",
        "RHI100210D" : "Int32",
        "RHI200210D" : "Int32",
        "RHI300210D" : "Int32",
        "RHI400210D" : "Int32",
        "RHI500210D" : "Int32",
        "RHI600210D" : "Int32",
        "RHI700210D" : "Int32",
        "RHI800210D" : "Int32"
    }
    columns = {
        "Areaname" : "Areaname",                               # County, State
        "STCOU" : "FIPS",                                      # FIPS
        "RHI100210D" : "White_Alone",                          # Resident population: White alone (April 1 - complete count) 2010
        "RHI200210D" : "Black_Alone",                          # Resident population: Black alone (April 1 - complete count) 2010
        "RHI300210D" : "AmericanIndian_AlaskaNative_Alone",    # Resident population: American Indian and Alaska Native alone (April 1 - complete count) 2010
        "RHI400210D" : "Asian_Alone",                          # Resident population: Asian alone (April 1 - complete count) 2010
        "RHI500210D" : "PacificIslander_Alone",                # Resident population: Native Hawaiian and Other Pacific Islander alone (April 1 - complete count) 2010
        "RHI600210D" : "2+_Races",                             # Resident population: Two or more races (April 1 - complete count) 2010
        "RHI700210D" : "Hispanic_Latino_Origin",               # Resident population: Hispanic or Latino Origin (April 1 - complete count) 2010
        "RHI800210D" : "Not_Hispanic_White_Alone"               # Resident population: Not Hispanic, White alone (April 1 - complete count)  2010
    }
    data = pd.read_csv("./datasets/USCensus/RHI_modified.csv", sep=",", header=0, engine="python", dtype=dtype, thousands=",").rename(columns=columns)
    # make PCT columns for relative race diversity
    # 0. list columns for all races
    race_columns = list(columns.values())[2:]
    # 1. get total calculated population for each county
    total_population = data[race_columns].sum(axis="columns")
    # 2. PCT = race population / total population
    for column in race_columns:
        data["PCT_" + column] = data[column] / total_population
    return data

In [ ]:
# NYTimes
virus_data = get_virus_data()

# USCensus data is completely organized, so simply combine the datasets
# merges all USCensus data into one dataframe
def init_census_data():
    df_list = list()
    df_list.append(get_land_data())
    df_list.append(get_housing_data())
    df_list.append(get_age_data())
    df_list.append(get_race_data())
    
    # merge data
    data = df_list[0]
    for i in range(1, len(df_list)):
        # ensure counties are in identical order
        if not (data["FIPS"] == df_list[i]["FIPS"]).all():
            print("County order mismatch")
            return None
        
        # if matches, remove "FIPS" and "Areaname" columns from all but the first
        df_list[i].drop(columns=["FIPS", "Areaname"], inplace=True)
        
        # merge into "data" dataframe
        data = pd.concat([data, df_list[i]], axis="columns")
    
    # split county and state names into separate columns
    new = data["Areaname"].str.split(pat=", ", n=1, expand=True)
    data["County"] = new[0].apply(lambda s: s + " County" if s not in EXCEPTIONS else s)  # add " County" if county
    data["State"] = new[1]
    data.drop(columns=["Areaname"], inplace=True)
    
    # remove state rows (i.e. rows that describe the entire state)
    # aka rows where the "State" column is now null due to split()
    data = filter_by_value(data, filter_dict={"County": EXCEPTIONS}, remove_dict={"State": "NaN"})
    return data

# USDoA data is **almost** entirely organized, so combine those together
# merges all USDoA data into one dataframe
def init_DoA_data():
    df_list = list()
    df_list.append(get_education_data())
    df_list.append(get_population_data())
    df_list.append(get_poverty_data())
    df_list.append(get_unemployment_data())
    
    # find any counties that don't exist in all datasets
    data = df_list[0]
    FIPS_keep = set(data["FIPS"].tolist())   # intersecting FIPS
    FIPS_remove = set()                      # symmetric difference FIPS
    for i in range(1, len(df_list)):
        FIPS_remove = FIPS_remove.union(FIPS_keep ^ set(df_list[i]["FIPS"].tolist()))  # FIPS that don't exist in both
        FIPS_keep = FIPS_keep.intersection(set(df_list[i]["FIPS"].tolist()))           # FIPS that exist in both
    # remove the counties that don't exist in all datasets
    for i in range(len(df_list)):
        i_remove = set()
        for fips in FIPS_remove:
            i_remove = i_remove.union(set(df_list[i][df_list[i]["FIPS"] == fips].index))
        df_list[i].drop(index=i_remove, inplace=True)
    
    # merge data
    data.reset_index(drop=True, inplace=True)
    for i in range(1, len(df_list)):
        df_list[i].reset_index(drop=True, inplace=True)
        # ensure counties are in identical order
        if not (data["FIPS"] == df_list[i]["FIPS"]).all():
            print("County order mismatch")
            return None
        
        # remove "State", "County", and "FIPS" columns from all but first
        df_list[i].drop(columns=["State", "County", "FIPS"], inplace=True)
        
        # merge into "data" dataframe
        data = pd.concat([data, df_list[i]], axis="columns")
    
    return data

census_data = init_census_data()
doa_data = init_DoA_data()

In [ ]:
# merges all data together based on their respective counties
def init_attributes():
    df_list = list()
    df_list.append(init_DoA_data())
    df_list.append(init_census_data())
    
    # find any counties that don't exist in all datasets
    data = df_list[0]
    FIPS_keep = set(data["FIPS"].tolist())   # intersecting FIPS
    FIPS_remove = set()                      # symmetric difference FIPS
    for i in range(1, len(df_list)):
        FIPS_remove = FIPS_remove.union(FIPS_keep ^ set(df_list[i]["FIPS"].tolist()))  # FIPS that don't exist in both
        FIPS_keep = FIPS_keep.intersection(set(df_list[i]["FIPS"].tolist()))           # FIPS that exist in both
    # remove the counties that don't exist in all datasets
    for i in range(len(df_list)):
        i_remove = set()
        for fips in FIPS_remove:
            i_remove = i_remove.union(set(df_list[i][df_list[i]["FIPS"] == fips].index))
        df_list[i].drop(index=i_remove, inplace=True)
        
    # merge data
    data.reset_index(drop=True, inplace=True)
    for i in range(1, len(df_list)):
        df_list[i].reset_index(drop=True, inplace=True)
        # ensure counties are in identical order
        if not (data["FIPS"] == df_list[i]["FIPS"]).all():
            print("County order mismatch")
            return None
        
        # remove "State", "County", and "FIPS" columns from all but first
        df_list[i].drop(columns=["State", "County", "FIPS"], inplace=True)
        
        # merge into "data" dataframe
        data = pd.concat([data, df_list[i]], axis="columns")
    
    return data
    
attributes_data = init_attributes()

In [ ]:
# NOTE: As far as I can tell, the COVID19 data from NYTimes seems to consolidate 5 counties under New York City (and thus it has no listed FIPS code)
#       as a result, I will have to simply link the 5 counties (Manhattan/New York, Queens, Bronx, Brooklyn/Kings, Staten Island/Richmond).
#       For more information: https://github.com/nytimes/covid-19-data/issues/105
# get FIPS codes corresponding to the 5 NYC counties
nyc_FIPS = [int(fips) for fips in list(filter_by_value(attributes_data, filter_dict={"State": "NY", "County": ["Bronx County", "Kings County", "New York County", "Queens County", "Richmond County"]})["FIPS"])]
print(nyc_FIPS)

In [ ]:
# combines all given records into one (i.e. absolute values can be summed, but percentages and rates need to be recalculated individually)
# precondition: for all percentage and rate columns (prefix "PCT_" and "R_" respectively), there is a corresponding absolute value column of the same name without the prefix
def combine_counties(data, combined_FIPS: list = [], fill_data: dict = {}):
    if len(combined_FIPS) == 0:
        return data
    
    # ensure necessary data exists in the dataframe
    relevant_data = filter_by_value(data, filter_dict={"FIPS": combined_FIPS})
    if relevant_data.empty:
        return data
    
    # ensure records that match fill_data don't already exist
    if not filter_by_value(data, filter_dict=fill_data).empty:
        return data
    
    dtypes = dict(data.dtypes)    # save old types to reconvert new dataframe (they all get casted to float for some reason)
    combined = fill_data
    # for each column, sum if is absolute value, recalculate if is percentage or rate
    for column in data.columns:
        # skip the columns already filled
        if column in combined.keys():
            continue
        
        # if column is a rate
        if column[0:2] == "R_":
            # rates are defined as value per thousand people per year
            # i.e. a 10 birth rate is 10 births per 1000 people per year
            # i.e. a 10 birth rate in a 10000 person population is 10 * 10000 / 1000 = 100 births per year
            # use "POP_ESTIMATE_2018" column as given population to calculate combined rates
            # new birth rate = sum(births) * 1000 / sum(population)
            sum_subtotal = relevant_data[column.split("R_")[1]].sum(axis="index")
            combined[column] = sum_subtotal * 1000 / relevant_data["POP_ESTIMATE_2018"].sum(axis="index")
            dtypes[column] = "float64"    # ensure float type
        # if column is a percentage
        elif column[0:4] == "PCT_":
            # new PCT = sum(subtotal) / sum(total)
            # sum(total) = sum(subtotal / subPCT)
            sum_subtotal = relevant_data[column.split("PCT_")[1]].sum(axis="index")
            sum_total = (relevant_data[column.split("PCT_")[1]] / relevant_data[column]).sum(axis="index")
            combined[column] = sum_subtotal / sum_total
            dtypes[column] = "float64"    # ensure float type
        # if column is an absolute value
        else:
            combined[column] = relevant_data[column].sum(axis="index")
    
    data = data.append(combined, ignore_index=True)
    # convert back to original types
    for column in data.columns:
        data[column] = data[column].astype(dtypes[column])
    
    return data.sort_values(by="FIPS", axis="index")

# group the 5 NYC counties
attributes_data = combine_counties(attributes_data, combined_FIPS=nyc_FIPS, fill_data={"FIPS": 36000, "State": "NY", "County": "NYC"})
# delete the 5 individual counties
attributes_data = filter_by_value(attributes_data, remove_dict={"FIPS": nyc_FIPS})
# remove other counties with unknown FIPS from virus data EXCEPT for NYC
virus_data = filter_by_value(virus_data, filter_dict={"County": "New York City"}, remove_dict={"FIPS": -1})

In [ ]:
# change NYC FIPS to 36000 in data if necessary
def apply_FIPS(data, fips: int = -1, condition_dict: dict = {}):
    df = data.copy()
    for col, value in condition_dict.items():
        df.loc[df[col] == value, "FIPS"] = fips
    return df
        
virus_data = apply_FIPS(virus_data, fips=36000, condition_dict={"County": "New York City"})

In [ ]:
attributes_data

In [ ]:
virus_data

In [ ]:
# merge census/DoA data into virus data
def merge_data(a_data, v_data):
    data = a_data.copy()
    virus_data = v_data.copy()
    dtypes = dict(data.dtypes)
    dtypes.update(dict(virus_data.dtypes))
    
    # 1. remove "County" and "State" columns from virus_data to avoid repeats
    virus_data.drop(columns=["County", "State"], inplace=True)
    
    # 2. remove counties that don't exist in both datasets
    data_remove_index = set()
    virus_remove_index = set()
    # for fips that don't exist in both datasets:
    for fips in set(data["FIPS"]) ^ set(virus_data["FIPS"]):
        data_remove_index = data_remove_index.union(data[data["FIPS"] == fips].index)
        virus_remove_index = virus_remove_index.union(virus_data[virus_data["FIPS"] == fips].index)
    data.drop(index=data_remove_index, inplace=True)
    virus_data.drop(index=virus_remove_index, inplace=True)
    
    # 3. extend each virus record with attributes data, then append to list
    print("Starting...")
    start_time = time.time()
    cycle_time = start_time
    combined_list = []
    iterations = 0
    cycles = 0
    cycle_count = 50000
    for fips in set(virus_data["FIPS"]):            
        attributes = data[data["FIPS"] == fips]
        if attributes.empty:
            print("Attributes for county {} not found, exiting...".format(fips))
            break
        attributes = attributes.drop(columns="FIPS").iloc[0]
        virus_records_by_FIPS = virus_data[virus_data["FIPS"] == fips]
        # add same attributes to all entries of this county
        attributes_df = pd.DataFrame([attributes for _ in range(len(virus_records_by_FIPS))], index=virus_records_by_FIPS.index)
        new_records = pd.concat([virus_records_by_FIPS, attributes_df], axis="columns")        
        combined_list.append(new_records)
        
        iterations += len(virus_records_by_FIPS)
        if iterations > cycle_count:
            prev_time = cycle_time
            cycle_time = time.time()
            print("Iteration {} in {} seconds".format(cycles*cycle_count + iterations, cycle_time - prev_time))
            iterations -= cycle_count
            cycles += 1
    print("{} records extended in {} seconds".format(cycles*cycle_count + iterations, time.time() - start_time))
    print("{} counties kept".format(len(set(virus_data["FIPS"]))))
    
    # 4. turn list of dataframes into one dataframe
    df_time = time.time()
    combined = pd.DataFrame(pd.concat(combined_list, axis="index"))
    print("DataFrame created in {} seconds".format(time.time() - df_time))
    
    # 5. convert back to original types
    for column in combined.columns:
        combined[column] = combined[column].astype(dtypes[column])
    
    return combined.sort_index()

data = merge_data(attributes_data, virus_data)

In [ ]:
data[data["County"] == "San Mateo County"]

In [ ]:
# split data based on "by" parameter, returns dictionary of dataframes split by key
def split_data(data, by: str) -> dict:
    if not by in data.columns:
        print("Column not found, exiting...")
        return None
    
    split_dict = dict()
    for value in set(data[by]):
        split_dict[value] = data[data[by] == value]
    return split_dict
    
data_by_state = split_data(data, by="State")

In [ ]:
len(set(data_by_state["HI"]["FIPS"]))

In [ ]:
# split data into features and target(s)
def feature_target_split(df, targets: list, drops: list = list()):
    if len(targets) == 0:
        print("No target specified, exiting...")
        return None
    
    data = df.copy()
    
    # ensure all columns to be dropped exist
    for drop in drops:
        if not drop in data.columns:
            print("Column {} not found, can't be dropped, exiting...".format(drop))
            return None
    # drop specified columns
    data.drop(columns=drops, inplace=True)
    
    # separate targets
    targets_list = list()
    for target in targets:
        # ensure all targets exist
        if not target in data.columns:
            print("Target {} not found, exiting...".format(target))
            return None
        targets_list.append(data[target])
        
    # separate features
    features = data.drop(columns=targets)
    # if just one target, return by itself
    if len(targets_list) == 1:
        return features, targets_list[0]
    # if multiple targets, return as list
    return features, targets_list

In [ ]:
# normalize columns, omitting any specified columns
def normalize_columns(data, ignore: list = []):
    normalized = data.drop(columns=ignore)
    means = normalized.mean(axis="index", skipna=False)
    stds = normalized.std(axis="index", skipna=False)
    for column in normalized.columns:
        normalized[column] = normalized[column].sub(means[column]).div(stds[column])
    return normalized

In [ ]:
# print weight values with their labels
# if top specified, print highest "top" weights; if not, print all
def output_weights(labels, weights, top: int = 0, console: bool = False):
    if len(labels) != len(weights):
        print("Length of labels and weights don't match, exiting...")
        return None
    
    if console: print("     -----")
    sorted_weights = sorted(zip(abs(weights), labels, list(range(len(weights)))), reverse=True)        
    if top <= 0:
        for i in range(len(sorted_weights)):
            if console: print("{}: {}".format(sorted_weights[i][1], weights[sorted_weights[i][2]]))
            top_list.append((sorted_weights[i][1], weights[sorted_weights[i][2]]))
    else:
        if top > len(weights):
            top = len(weights)
        
        top_list = list()
        if console: print("Top {} weights:".format(top))
        for i in range(top):
            if console: print("{}: {}".format(sorted_weights[i][1], weights[sorted_weights[i][2]]))
            top_list.append((sorted_weights[i][1], weights[sorted_weights[i][2]]))
    if console: print("     -----")
    return top_list

In [ ]:
# performs linear regression on data
# cycle_duration: split data off into sections spanning cycle_duration (in days) and fit model
# regression_type: False -> linear, True -> ridge
def regression(data, regression_type: bool = False, filename: str = "", cycle_duration: int = 7, start_day: int = -1, top: int = 10, console: bool = False):
    if type(data) is dict:
        # parallel arrays to write data to csv file
        categories = list()
        train_cases_scores = list()
        train_deaths_scores = list()
        test_cases_scores = list()
        test_deaths_scores = list()
        top_cases_weights = list()
        top_deaths_weights = list()
        records_used = list()
        counties_used = list()
        
        start_time = time.time()
        for category in sorted(data.keys()):
            # TODO: add number of different counties used
            categories.append(category)
            df = data[category]
            if console:
                print("START: {}".format(category))
                print("--------------------")
            # set starting day of first cycle
            max_day = max(df["Day"])
            min_day = max(min(df["Day"]), start_day)
            start_day = ((max_day + 1 - min_day) % cycle_duration) + min_day
            # list of dataframes split into their respective cycles
            df_cycles = [filter_by_value(df, filter_dict={"Day": list(range(day, day + cycle_duration))}) for day in range(start_day, max_day+1, cycle_duration)]
            
            for cycle, df_cycle in enumerate(df_cycles):
                if console:
                    print("   -----")
                    print("Cycle {}: [Days {}~{}]".format(cycle, min(df_cycle["Day"]), min(df_cycle["Day"])+cycle_duration-1))
                X, [Y_Cases, Y_Deaths] = feature_target_split(df_cycle, targets=["Cases", "Deaths"])
                X = normalize_columns(X, ignore=["Date", "FIPS", "State", "County"])
                X_train, X_test, Y_train_Cases, Y_test_Cases, Y_train_Deaths, Y_test_Deaths = \
                    train_test_split(X, Y_Cases, Y_Deaths, test_size=0.2, random_state=42)
                
                if regression_type:
                    reg_Cases = Ridge().fit(X_train, Y_train_Cases)
                    reg_Deaths = Ridge().fit(X_train, Y_train_Deaths)
                else:
                    reg_Cases = LinearRegression().fit(X_train, Y_train_Cases)
                    reg_Deaths = LinearRegression().fit(X_train, Y_train_Deaths)
                
                train_cases_scores.append(reg_Cases.score(X_train, Y_train_Cases))
                train_deaths_scores.append(reg_Deaths.score(X_train, Y_train_Deaths))
                test_cases_scores.append(reg_Cases.score(X_test, Y_test_Cases))
                test_deaths_scores.append(reg_Deaths.score(X_test, Y_test_Deaths))
                if console:
                    print("Training score (cases): {}".format(train_cases_scores[-1]))
                    print("Training score (deaths): {}".format(train_deaths_scores[-1]))
                    print("Test score (cases): {}".format(test_cases_scores[-1]))
                    print("Test score (deaths): {}".format(test_deaths_scores[-1]))
                top_cases_weights.append(output_weights(X.columns, reg_Cases.coef_, top, console))
                top_deaths_weights.append(output_weights(X.columns, reg_Deaths.coef_, top, console))
                records_used.append(len(df_cycle))
                counties_used.append(len(set(df_cycle["FIPS"])))
                if console: print("   -----")
            if console: print("--------------------")
        print("Finished in {} seconds".format(time.time() - start_time))
        if len(filename) > 0:
            with open("./outputs/" + filename, mode="w", newline="") as logs:
                logger = csv.writer(logs, delimiter=",")
                logger.writerow(["CATEGORY"] + categories)
                logger.writerow(["TRAIN CASES SCORES"] + train_cases_scores)
                logger.writerow(["TRAIN DEATHS SCORES"] + train_deaths_scores)
                logger.writerow(["TEST CASES SCORES"] + test_cases_scores)
                logger.writerow(["TEST DEATHS SCORES"] + test_deaths_scores)
                # TODO: this is kind of ugly, rewrite if possible
                for num in range(top):
                    top_num_row = list()
                    for cat in range(len(categories)):
                        top_num_row.append(top_cases_weights[cat][num])
                    logger.writerow(["TOP CASES WEIGHT #{}".format(num+1)] + top_num_row)
                for num in range(top):
                    top_num_row = list()
                    for cat in range(len(categories)):
                        top_num_row.append(top_deaths_weights[cat][num])
                    logger.writerow(["TOP DEATHS WEIGHT #{}".format(num+1)] + top_num_row)
                logger.writerow(["Records Used"] + records_used)
                logger.writerow(["Counties Used"] + counties_used)
                
    # if not dictionary
    else:
#         print("Not implemented yet! Exiting...")
        return regression({"US": data}, regression_type, filename, cycle_duration, start_day, top, console)
        
    return (categories, train_cases_scores, train_deaths_scores, test_cases_scores, test_deaths_scores, top_cases_weights, top_deaths_weights)

regression(data_by_state, regression_type=False, filename="linear_regression_by_state.csv", cycle_duration=30, start_day=100)
regression(data, regression_type=False, filename="linear_regression.csv", cycle_duration=30, start_day=100)
regression(data_by_state, regression_type=True, filename="ridge_regression_by_state.csv", cycle_duration=30, start_day=100)
regression(data, regression_type=True, filename="ridge_regression.csv", cycle_duration=30, start_day=100)
regression(data, regression_type=False, filename="linear_regression_1_day.csv", cycle_duration=2, start_day=130, console=True)
regression(data, regression_type=True, filename="ridge_regression_1_day.csv", cycle_duration=2, start_day=130, console=True)